In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, t

#### Value at Risk and Conditional Value at Risk Engine Overview

In this project, I explore the different methods of calculating VaR and CVaR.

In [22]:
# Load prices from adj close prices csv and computing log returns
prices = pd.read_csv("adj_close_prices.csv", parse_dates=["Date"], index_col="Date").sort_index()
log_returns = np.log(prices / prices.shift(1)).dropna()

In [24]:
# Generating random portfolio weights
weights = np.random.random(len(log_returns.columns))
weights /= np.sum(weights)

In [26]:
# Compute covariance matrix
cov_mat = log_returns.cov()

In [28]:
# Historical VaR
def Historical_VaR(weights, log_returns, amount_invested, confidence, lookback_days=520):
    recent_returns = log_returns[-lookback_days:]
    portfolio_returns = recent_returns.dot(weights)
    percentile = (1 - confidence) * 100
    hist_Var = -np.percentile(portfolio_returns, percentile) * amount_invested
    return hist_Var

#### Historical VaR
Historical Value at Risk 

In [34]:
# Parametric (Normal) VaR
def Parametric_VaR(cov_mat, weights, portfolio_value, confidence):
    portfolio_std = np.sqrt(weights.T @ cov_mat.values @ weights)
    z_score = norm.ppf(1 - confidence)
    one_day_VaR = -z_score * portfolio_std * portfolio_value
    return one_day_VaR

#### Parametric VaR


In [45]:
# Parametric t-distribution VaR
def Tdist_Parametric_VaR(cov_mat, weights, portfolio_value, confidence, df=5):
    portfolio_std = np.sqrt(weights.T @ cov_mat.values @ weights)
    t_score = t.ppf(1 - confidence, df)
    one_day_VaR = -t_score * portfolio_std * portfolio_value
    return one_day_VaR

#### Parametric VaR under the T distribution

In [47]:
# Testing
amount_invested = 1000000
confidence = 0.95

hist_var = Historical_VaR(weights, log_returns, amount_invested, confidence)
param_var = Parametric_VaR(cov_mat, weights, amount_invested, confidence)
t_var = Tdist_Parametric_VaR(cov_mat, weights, amount_invested, confidence)

print(f"Historical VaR (95%): ${hist_var:,.2f}")
print(f"Parametric VaR (Normal, 95%): ${param_var:,.2f}")
print(f"t VaR (df=5, 95%): ${t_var:,.2f}")

Historical VaR (95%): $13,649.12
Parametric VaR (Normal, 95%): $14,939.26
t VaR (df=5, 95%): $18,301.53


In [49]:
def Historical_CVaR(weights, log_returns, amount_invested, confidence, lookback_days=520):
    recent_returns = log_returns[-lookback_days:]
    portfolio_returns = recent_returns.dot(weights)
    var_threshold = np.percentile(portfolio_returns, (1 - confidence) * 100)

    # Get all returns worse than or equal to the VaR threshold
    tail_losses = portfolio_returns[portfolio_returns <= var_threshold]

    # Average tail loss × invested capital
    cvar = -np.mean(tail_losses) * amount_invested
    return cvar


In [51]:
def Parametric_CVaR(cov_mat, weights, portfolio_value, confidence):
    portfolio_std = np.sqrt(weights.T @ cov_mat.values @ weights)
    z = norm.ppf(confidence)
    cvar = portfolio_value * portfolio_std * norm.pdf(z) / (1 - confidence)
    return cvar


In [53]:
amount_invested = 1_000_000
confidence = 0.95

hist_cvar = Historical_CVaR(weights, log_returns, amount_invested, confidence)
param_cvar = Parametric_CVaR(cov_mat, weights, amount_invested, confidence)

print(f"Historical CVaR (95%): ${hist_cvar:,.2f}")
print(f"Parametric CVaR (95%): ${param_cvar:,.2f}")


Historical CVaR (95%): $19,072.07
Parametric CVaR (95%): $18,734.44


In [55]:
def MonteCarlo_VaR(log_returns, weights, amount_invested, confidence=0.95, num_simulations=10000):
    # Mean and covariance matrix of returns
    mean = log_returns.mean().values
    cov = log_returns.cov().values

    # Simulate future returns using multivariate normal distribution
    simulated_returns = np.random.multivariate_normal(mean, cov, size=num_simulations)

    # Simulate portfolio returns
    portfolio_returns = simulated_returns @ weights

    # Calculate losses
    portfolio_losses = -portfolio_returns * amount_invested

    # Calculate the VaR at the (1 - confidence) percentile
    var = np.percentile(portfolio_losses, (1 - confidence) * 100)

    return var


In [57]:
def MonteCarlo_CVaR(log_returns, weights, amount_invested, confidence, num_simulations=10000):
    # Compute mean and covariance matrix
    mean = log_returns.mean().values
    cov = log_returns.cov().values
    
    # Simulate future returns (1-day) from multivariate normal
    simulated_returns = np.random.multivariate_normal(mean, cov, size=num_simulations)
    
    # Portfolio returns = dot product of simulated asset returns with weights
    portfolio_returns = simulated_returns @ weights

    # Compute portfolio losses
    portfolio_losses = -portfolio_returns * amount_invested

    # Compute VaR threshold
    var_cutoff = np.percentile(portfolio_losses, (1 - confidence) * 100)

    # CVaR = mean of all losses worse than or equal to VaR
    cvar = portfolio_losses[portfolio_losses >= var_cutoff].mean()

    return var_cutoff, cvar
